In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as mpl_anim  
from IPython.display import HTML
from qutip import *

# Parameters
N = 40
w = 1.0
g = 0.05
psi0 = tensor(basis(N, 0), basis(2, 0)) 

# Hamiltonian
a  = tensor(destroy(N), qeye(2))
sm = tensor(qeye(N), sigmam())
sz = tensor(qeye(N), sigmaz())
H = w * a.dag() * a + 0.5 * w * sz + g * (a.dag() * sm + a * sm.dag())

# Time Evolution (One half-Rabi flop: Vacuum -> 1 Photon)
t_max = np.pi / (2 * g)
frames = 60 # Number of frames in the animation
tlist = np.linspace(0, t_max, frames)

print("1. Running Simulation")
output = mesolve(H, psi0, tlist, [], [])

# Pre-Calculate Wigner Data 
print("2. Calculating Wigner frames")

xvec = np.linspace(-5, 5, 100)
wigner_data = []

# Loop through every stored state
for state in output.states:
    rho_cavity = state.ptrace(0)        
    W = wigner(rho_cavity, xvec, xvec)  
    wigner_data.append(W)

# Build the Animation
print("3. Building Animation")

fig, ax = plt.subplots(figsize=(6, 6))

w_max = np.max(wigner_data[0])
w_min = -w_max 

# Initial Plot
mesh = ax.contourf(xvec, xvec, wigner_data[0], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))

ax.set_xlabel("Position (q)")
ax.set_ylabel("Momentum (p)")
ax.set_title(f"Atom Emitting Photon (t=0.00)")
fig.colorbar(mesh, ax=ax, label="Quasiprobability")

# Update Function
def update(frame):
    ax.clear() # Clear previous frame
    
    # Redraw Wigner function
    ax.contourf(xvec, xvec, wigner_data[frame], 100, cmap="RdBu_r", levels=np.linspace(w_min, w_max, 100))
    
    # Redraw labels
    ax.set_xlabel("Position (q)")
    ax.set_ylabel("Momentum (p)")
    ax.set_title(f"Atom Emitting Photon (t={tlist[frame]:.2f})")
    return ax,

anim = mpl_anim.FuncAnimation(fig, update, frames=frames, interval=100, blit=False)
plt.close() # Prevents showing the static plot twice
HTML(anim.to_jshtml())

1. Running Simulation...
2. Calculating Wigner frames (this might take 10-20 seconds)...


c:\Users\Vedant\miniconda3\envs\qiskit-ok\lib\site-packages\qutip\solver\solver_base.py:598: FutureWarning: e_ops will be keyword only from qutip 5.3 for all solver
  warnings.warn(


3. Building Animation...
